# Tutorial: How to set-up a new isolated nbdev development environment in Ubuntu

> A tutorial on how to set-up a new isolated Ubuntu nbdev environment with chroot and python virtual environment.

In this tutorial, you will install a clean isolated development environment for python, Jupyter notebook and [nbdev](https://nbdev.fast.ai/).

You will use many powerful features such as: linux chroot, python virtual environment, Jupyter notebook and nbdev.

## Prerequisites

You need a computer running the latest version of Ubuntu. This tutorial is written for Ubuntu version 22, named *Jammy*. Although it is recommended to use the same version, most of the tutorial can be followed with other versions of Ubuntu.

Make sure to install: debootstrap, schroot.

You can install them with:

```sh
sudo apt install debootstrap
sudo apt install schroot
```

You need a local Linux user - in this tutorial, mine is called ```chris```.

And you also need a github account, and an [authentication token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token) already configured.


## Create a chroot environment

### Initiate the chroot with debootstrap

You will first create a clean chroot environment. A chroot is an isolated linux environment where packages are installed independently of the main operating system packages. Whatever is installed in the chroot will not contaminate your main operating system.

::: {.callout-note}
If you want to know more about chroot, you can read the [wikipedia article](https://en.wikipedia.org/wiki/Chroot).
:::

```sh
sudo debootstrap jammy nbdev
```

### Mount bind OS folder

Bind mount dev, proc and sys in the chroot env:

```sh
sudo mount --rbind /dev ~/mychroots/nbdev/dev
sudo mount --rbind /proc ~/mychroots/nbdev/proc
sudo mount --rbind /sys ~/mychroots/nbdev/sys
```


### Copy apt source configuration 

Copy the sources apt repo to the chroot env:

```sh
sudo cp /etc/apt/sources.list ~/mychroots/nbdev/etc/apt/sources.list
```

You will be able to install any package in the chroot environment from the repository list.

### Edit schroot configuration

Edit schroot configuration as follows for the new chroot environment:

```sh
sudo vi /etc/schroot/schroot.conf
```
```
[nbdev]
description=nbdev - Jammy
directory=/home/chris/mychroots/nbdev1
users=chris
root-groups=root
```

### Configure the user in the chroot environment

Create the first user (I recommend to use the same user as the one doing the schroot in the mother environment) and add it to sudoers.
For convenience, I configure it to run without password when doing sudo:

```sh
adduser chris
usermod -aG sudo chris
visudo
```
Add the following line:
```
	  chris ALL=(ALL) NOPASSWD:ALL
```

Now you are done with the chroot environment ! 

You can simply enter the chroot environment with your user as follows:

```sh
schroot -c nbdev --directory /home/chris
```

To exit the environment, type simply:
```sh
exit
```

## Packages Installation

### Upgrade

You start by upgrading your local packages with the latest updates:

```sh
sudo apt update
sudo apt upgrade
```

### Packages installation

Then, you install the new packages needed for your environment:

```sh
sudo apt install python3-pip python3-venv git
```

### Install github client

You have to copy and paste the following command:

```sh
type -p curl >/dev/null || sudo apt install curl -y
curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg \
&& sudo chmod go+r /usr/share/keyrings/githubcli-archive-keyring.gpg \
&& echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null \
&& sudo apt update \
&& sudo apt install gh -y
```

## Configure git and github

### Git configuration

For git, you configure your user name and email - so that your commit messages include the right author:

```sh
git config --global user.email YOUR_EMAIL
git config --global user.name YOUR_NAME
```

### Github configuration

For Github, you need to register your authentication token.

```
gh auth login
```

Follow the prompts, select ```Github.com``` and as method for authentication: ```with authentication token```.

Note that your authentication token will expire after some time, and you will need to configure again your github access from time to time.

## Install your Python virtual environment

Still inside the chroot environment, cd to your home directory, and build a python virtual environment. Then install ```jupyter, jupyter_contrib_nbextensions, jupyterthemes``` and ```nbdev```

```sh
cd
mkdir pyenv
cd pyenv
python3 -m venv nbenv
source nbenv/bin/activate
python3 -m pip install jupyter jupyter_contrib_nbextensions jupyterthemes nbdev
```

## Configure Jupyter notebook

You then enable 2 useful jupyter notebook extensions:

```sh
jupyter nbextension enable collapsible_headings/main
jupyter nbextension enable toc2/main
```

And you enable a Jupyter dark theme - I like ```cheterish```:

```sh
jt -t chesterish
```

## Your first nbdev project


### Initiate your github repository

You create a new github repository. 
Once this is done, copy its HTTPS clone url.

Clone your repository in your development environment:

```sh
git clone YOUR_REPOSITORY_HTTPS_URL
```

### Configure the nbdev environment

Now you can set-up the initial nbdev environment with:

```sh
nbdev_new
nbdev_install_hooks
git add .
git commit -m 'Creation'
git push
```

### Open the Jupyter notebook

Within your project folder structure, you can start Jupyter and start working with the notebook in the ```nbs``` folder:

```sh
jupyter notebook nbs/
```

## Congratulations 

Congratulations you have a working nbdev environment in a completely isolated environment with chroot and python virtual environment ! You can now enjoy the power of Jupyter notebooks and nbdev library.

## Next Steps

You can now continue with the [nbdev walkthough](https://nbdev.fast.ai/tutorials/tutorial.html). 

You can explore the nbdev features: github pages workflow integration, pypi integration, etc.

Enjoy coding with nbdev !